In [ ]:
# ======================================================
# Nota: Los datos ya deben estar ordenados de acuerdo al fofid
# =======================================================
import numpy as np
import time
import matplotlib.pyplot as plt
import os
import cuts

In [ ]:
path="/home/esteban/Escritorio/Practica/Data/"
path+="G13_millennium"  # Guo2013 
#path = "/home/esteban/Escritorio/Practica/Data/G14_millennium" # Gonzalez2014 
#path = "/home/esteban/Escritorio/Practica/Data/L16_millennium" # Lacey2016 
os.chdir( path )

# Loading database
x = np.load('x.npy')
fofid = np.load('fofid.npy')
stellarmass = np.load('stellarmass.npy')
centralmvir  = np.log10(np.load('centralmvir.npy') + 1e-10) + 10
type_gal = np.load('type.npy')
sfr = np.load('sfr.npy')

# Density samples (Only 1 run!) Done!
#cuts.density_data(sfr, stellarmass)

In [ ]:
log_centralmvir = centralmvir
logm_min, logm_max = 10, 15.
NBIN = 30

labels = ((log_centralmvir - logm_min)/(logm_max - logm_min) * NBIN).astype(int)

mask = (labels >= NBIN) | (labels < 0)
fofid = fofid[~mask]
stellarmass = stellarmass[~mask]
log_centralmvir = log_centralmvir[~mask]
type_gal = type_gal[~mask]
sfr = sfr[~mask]
labels = labels[~mask]
x = x[~mask]

bin_width = (logm_max - logm_min)/NBIN
bins = np.array([logm_min + bin_width*i for i in range(NBIN)]) + bin_width *.5

In [ ]:
start_time = time.time()
# Density selection
# =================
#   Fila    log(rho)
#    15      -2.5
#    10      -3.0
#     6      -3.5
# =================
log_rho, mass_cuts, sfr_cuts = np.loadtxt('density_data.txt', unpack = True)
rho, mass_cut, sfr_cut = log_rho[15], mass_cuts[15], sfr_cuts[15]

# Centrals data
idx = np.where(type_gal == 0)[0]
fofid_ar = fofid[idx]
logcentralmvir_ar = log_centralmvir[idx]
labels_ar = labels[idx]
x_ar = x[idx]

# Number of central = number of halos
len_data = len(fofid_ar)

# To save halos data
All_ar = np.zeros(len_data)
Cent_ar = np.zeros(len_data)
Sats_ar = np.zeros(len_data)

i,j = 0,0
Nsats, Ncent, Nall = 0,0,0
len_fulldata = len(fofid)

# To make HODs
Len_Sats = np.zeros(NBIN)
Len_Cent = np.zeros(NBIN)
Len_All = np.zeros(NBIN)
Halos = np.zeros(NBIN)

display_step = len_data/10
while i < len_data:
    index = labels_ar[i]
    cond = (fofid[j] == fofid_ar[i])
    while cond:
        if stellarmass[j] > mass_cut: 
            Len_All[index] += 1
            Nall += 1
            if type_gal[j] != 0: 
                Len_Sats[index] += 1
                Nsats += 1
            else:
                Len_Cent[index] += 1 
                Ncent += 1
        j += 1
        if j == len_fulldata: break
        cond = (fofid[j] == fofid_ar[i])
        
    Halos[index] += 1  
    Cent_ar[i] = Ncent
    Sats_ar[i] = Nsats
    All_ar[i] = Nall
    Ncent, Nsats, Nall = 0,0,0
    i += 1
    if i % display_step == 0:
        print "Step: %i Time: %3fs" % (i, (time.time() - start_time))

In [ ]:
# Save data
data = np.array([fofid_ar, logcentralmvir_ar, All_ar, Cent_ar, Sats_ar, x_ar]).T
np.save("halos_masscut_%.1f" %rho, data)

In [ ]:
HOD = np.log10(Len_All / Halos + 1e-10)
Sats = np.log10(Len_Sats / Halos + 1e-10)
Cent = np.log10(Len_Cent / Halos + 1e-10)

data = np.array([bins, HOD, Cent, Sats]).T
np.save("hod_masscut_%.1f" %rho, data)

In [ ]:
line = np.zeros(NBIN)
f = plt.figure()
path = "/home/esteban/Escritorio/Practica/Plots/G13_millennium" # Guo2013 
#path = "/home/esteban/Escritorio/Practica/Data/G14_millennium" # Gonzalez2014 
#path = "/home/esteban/Escritorio/Practica/Data/L16_millennium" # Lacey2016 
os.chdir( path )

plt.plot(bins, HOD, 'k.', label = 'All')
plt.plot(bins, Cent, 'r--', label = 'Centrals')
plt.plot(bins, Sats , 'b--', label = 'Sats')
plt.plot(bins, line, 'k--', lw = .5 )
plt.title(r"$n = 10^{%.1f} /h^{-3} Mpc^3$" %rho)
plt.xlabel(r"$log(M_h / h^{-1}M_{\odot})$")
plt.ylabel(r"$log(N)$")
plt.ylim(-2,2)
plt.xlim(10,15)
plt.minorticks_on()
plt.legend(loc='upper left', frameon = False, prop={'size':10})
f.savefig('hod_03.pdf')